# Working with unstructured data (JSON)

In this task we are intereted in performing some basic analytics on music played by users. Data is formated in JSON.

In [77]:
import pandas as pd
from collections import OrderedDict

In [18]:
file_location = 'data/song/song.json'  # improting the file
df = pd.read_json(file_location)  # easily converting the  json to data frame
df['time_played'] = pd.to_datetime(df['time_played']) # converting date to datetime

In [15]:
df.head(2)

,id,song_played,time_played,user_id,user_sign_up_date,user_state
0,GOQMMKSQQH,Hey Jude,2015-06-11 21:51:35,122,2015-05-16,Louisiana
1,HWKKBQKNWI,We Can Work It Out,2015-06-06 16:49:19,3,2015-05-01,Ohio


In [16]:
df.shape

(4000, 6)

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 6 columns):
id                   4000 non-null object
song_played          4000 non-null object
time_played          4000 non-null datetime64[ns]
user_id              4000 non-null int64
user_sign_up_date    4000 non-null object
user_state           4000 non-null object
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 187.6+ KB


### problem 1
What are top 5 and bottom 5 states in terms of number of users (not plays)

In [60]:
from collections import defaultdict
user_state = df[['user_state','user_id']].drop_duplicates()  # same user might have many actions
sorted_state = user_state.groupby('user_state').user_id.count().sort_values(ascending = True)  # calcuating the number of users

print("Lowest active states are:")
for i in range(0,5):
    print("{}:\t{} number of users!".format(sorted_state.index[i], sorted_state[i]))
print("\n")
print("Highest active states are:")
for i in [len(sorted_state) + i for i in range(-5,0)[::-1]]:
    print("{}:\t{} number of users!".format(sorted_state.index[i], sorted_state[i]))
 

Lowest active states are:
Arizona:	1 number of users!
New Mexico:	1 number of users!
Connecticut:	1 number of users!
Idaho:	1 number of users!
Nebraska:	1 number of users!


Highest active states are:
New York:	23 number of users!
California:	21 number of users!
Texas:	15 number of users!
Pennsylvania:	9 number of users!
Ohio:	9 number of users!


### problem 2: what are top and bottom states in terms of user engagement
We will define user engagement as the average number of plays per day per user.

In [94]:
# first let's resample data to become in daily format
dates = df.time_played.dt.date  # days of measurements
num_days = len(dates)
sorted_state_dict = dict(sorted_state) # state users
activity_state = df.groupby('user_state').id.count() # state activities
activity_state_dict = dict(activity_state)  # bringing it into dictionary format
user_engagement = {}
for key in activity_state_dict.keys():
    user_engagement[key] = activity_state_dict[key] / sorted_state_dict[key] / num_days

user_engagement = OrderedDict(sorted(user_engagement.items(), key = lambda x: x[1]))
user_engagement = list(user_engagement.items())

print("Lowest enganged states are:")
for i in range(0,5):
    activity = round(user_engagement[i][1],4)
    state = user_engagement[i][0]
    print("{}:\t{} activity per user per day!".format(state, activity))
print("\n")
print("Highest engaged states are:")
for i in [len(user_engagement) + i for i in range(-20,0)[::-1]]:
    # states with more than 5 users can be consider engaged
    activity = round(user_engagement[i][1],4)
    state = user_engagement[i][0]
    if sorted_state[state] < 5: 
        continue
    print("{}:\t{} activity per user per day!".format(state, activity))

Lowest enganged states are:
Kansas:	0.002 activity per user per day!
Virginia:	0.0021 activity per user per day!
Minnesota:	0.0026 activity per user per day!
West Virginia:	0.0032 activity per user per day!
Indiana:	0.0034 activity per user per day!


Highest engaged states are:
Florida:	0.0064 activity per user per day!
North Carolina:	0.0064 activity per user per day!
Missouri:	0.0064 activity per user per day!
Ohio:	0.0058 activity per user per day!
Georgia:	0.0056 activity per user per day!
Maryland:	0.0056 activity per user per day!
Illinois:	0.0053 activity per user per day!
Louisiana:	0.0052 activity per user per day!


### problem 3 : list users that signed up first at each state
We are peforming nest queries in order to reach what we desire.

In [108]:
df.groupby(['user_state','user_id'], as_index=False).user_sign_up_date.min().\
groupby('user_state').first().sort_values('user_sign_up_date')

,user_id,user_sign_up_date
user_state,,
Alabama,5,2015-05-01
Texas,7,2015-05-01
Oregon,1,2015-05-01
Ohio,3,2015-05-01
North Carolina,2,2015-05-01
New Mexico,4,2015-05-01
New Jersey,6,2015-05-01
Pennsylvania,11,2015-05-02
New York,10,2015-05-02
